In [1]:
import pandas as pd
import dict_map as dm

df = pd.read_csv('Daten/KoepckeEigen/trainingdata.csv',escapechar="\\",sep=",",error_bad_lines=False,warn_bad_lines=False)

token2idx, idx2token = dm.get_dict_map(df, 'token')
tag2idx, idx2tag = dm.get_dict_map(df, 'tag')

df['Word_idx'] = df['Wort'].map(token2idx)
df['Tag_idx'] = df['Attribut'].map(tag2idx)
#print(df)

df_group = df.groupby(by = ['satzId'], as_index=False)['Wort', 'Attribut', 'Word_idx', 'Tag_idx'].agg(lambda x: list(x))
#df_group

<ipython-input-1-7c52213ee09e>:13: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_group = df.groupby(by = ['satzId'], as_index=False)['Wort', 'Attribut', 'Word_idx', 'Tag_idx'].agg(lambda x: list(x))


In [2]:
import pad_train_test as ptt
train_tokens, val_tokens, test_tokens, train_tags, val_tags, test_tags = ptt.get_pad_train_test_val(df_group, df, tag2idx)

train_tokens length: 675 
train_tags length: 675 
test_tokens length: 100 
test_tags: 100 
val_tokens: 225 
val_tags: 225


In [3]:
input_dim = len(list(set(df['Wort'].to_list())))+1
output_dim = 32
input_length = max([len(s) for s in df_group['Word_idx'].tolist()])
n_tags = len(tag2idx)
print('input_dim: ', input_dim, '\noutput_dim: ', output_dim, '\ninput_length: ', input_length, '\nn_tags: ', n_tags)

input_dim:  15937 
output_dim:  32 
input_length:  927 
n_tags:  7


In [4]:
import model as md
#choco install graphviz, pip install pydot
from tensorflow.keras.utils import plot_model 
model_bilstm_lstm = md.get_bilstm_lstm_model(input_dim, output_dim, input_length, n_tags)
#plot_model(model_bilstm_lstm)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 927, 32)           509984    
_________________________________________________________________
bidirectional (Bidirectional (None, 927, 64)           16640     
_________________________________________________________________
lstm_1 (LSTM)                (None, 927, 32)           12416     
_________________________________________________________________
time_distributed (TimeDistri (None, 927, 7)            231       
Total params: 539,271
Trainable params: 539,271
Non-trainable params: 0
_________________________________________________________________
C:\Users\thilo\AppData\Local\Programs\Python\Python39\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


In [9]:
import numpy as np
results = pd.DataFrame()
results['with_add_lstm'] = md.train_model(train_tokens, np.array(train_tags), model_bilstm_lstm)

1/1 [==============================] - 33s 33s/step - loss: nan - accuracy: 1.8379e-04 - val_loss: nan - val_accuracy: 1.3584e-04
